In [ ]:
!python -m pip install requests
!python -m pip install praw
!python -m pip install redis

!python -m pip install HTMLParser

In [60]:
import requests
from bs4 import BeautifulSoup
import sqlite3






URL = "https://tappedout.net/mtg-decks/search/?q=lagrella&format=edh&price_min=&price_max=&o=-date_updated&submit=Filter+results&p=2&page=2"

page = requests.get(URL)







In [88]:
con = sqlite3.connect("decks2.db")
prints = sqlite3.connect("AllPrintings.sqlite")


In [152]:



import concurrent.futures

from concurrent.futures import thread


def searchForLinks(q ,deck_format="edh", price_min="", price_max="", update=""):
  #https://tappedout.net/mtg-decks/search/
# ?
# q=lagrella
# format=edh
# price_min=
# price_max=
# o=-date_updated
# submit=Filter+results
# p=2
# page=2
  baseURL = "https://tappedout.net/mtg-decks/search/?q={q}&format={deck_format}&price_min={price_min}&price_max={price_max}&o=-date_updated{update}&submit=Filter+results&p={p}&page={page}"
  req_succes = True
  page = 1
  links = []
  while req_succes:
    print(baseURL.format(p=page, page=page, q=q, deck_format=deck_format, price_min=price_min,price_max=price_max, update=update))
    webp = requests.get(baseURL.format(p=page, page=page, q=q, deck_format=deck_format, price_min=price_min,price_max=price_max, update=update))
    if webp.status_code == 200:
      page += 1
      soup = BeautifulSoup(webp.content, "html.parser")
      h3 = soup.find_all("h3", class_="name deck-wide-header")
      for h in h3:
        print(h.find("a")["href"])
        links.append(h.find("a")["href"])
    else:
      req_succes = False
  return links


def saveLinks(links, sqlConnector):
  sqlCursor = sqlConnector.cursor()

  for link in links:
    resp = sqlCursor.execute("SELECT name, fetched FROM deck_names WHERE name=?", (link,)).fetchall()
    if resp == []:
      sqlCursor.execute("INSERT INTO deck_names values (?,?,?)", (0,link,0))
      sqlConnector.commit()
  sqlCursor.close()

def fetchDecks(sqlConnector):
  sqlCursor = sqlConnector.cursor()
  fetchLinks = sqlCursor.execute("SELECT name FROM deck_names WHERE fetched=0").fetchall()
  allLinkCount = len(fetchLinks)
  progress = 0
  for link in fetchLinks:
    progress = progress + 1
    print("{prog}/{all}".format(prog=progress,all=allLinkCount))
    deck = parseDeckList(getDeckList(link[0]),sqlConnector)
    card_expanded = []
    for card in deck:
      while card[1] != 0:
        card_expanded.append(card[0])
        card[1] = card[1]-1
    print(len(card_expanded))
    if len(card_expanded) == 100:  
      sqlCursor.execute("INSERT INTO decks('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99') VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", card_expanded )
      sqlConnector.commit()
      responseRowId = sqlCursor.execute("SELECT last_insert_rowid()")
      rowId = responseRowId.fetchone()
      sqlCursor.execute("UPDATE deck_names SET fetched=1, id = ? WHERE name = ?", (rowId[0],link[0]))
      sqlConnector.commit()
    else:
      sqlCursor.execute("DELETE FROM deck_names WHERE name = ?",(link[0],))

def fetchDecksMultithread(sqlConnector):
  sqlCursor = sqlConnector.cursor()
  fetchLinks = sqlCursor.execute("SELECT name FROM deck_names WHERE fetched=0").fetchall()
  allLinkCount = len(fetchLinks)
  progress = 0
  threadCount = 10
  while len(fetchLinks) >= 0:
    
    print("{prog}/{all}".format(prog=progress,all=allLinkCount))
    if threadCount > len(fetchLinks):
      threadCount = len(fetchLinks)

    threadLinks = fetchLinks[0:threadCount]
    fetchLinks = fetchLinks[threadCount-1:]
    progress = progress + threadCount
    with concurrent.futures.ThreadPoolExecutor() as executor:
      deckLists = []
      for link in threadLinks:
        deckLists.append(executor.submit(getDeckListMultithread,link))


      
    
      print("decklist: {lenght}".format( lenght=len(deckLists)))
      for rDeck in concurrent.futures.as_completed(deckLists):
        dDeck = rDeck.result()
        deck = parseDeckList(dDeck[1],sqlConnector)
        
        
        card_expanded = []
        for card in deck:
          while card[1] != 0:
            card_expanded.append(card[0])
            card[1] = card[1]-1
        print(len(card_expanded))
        if len(card_expanded) == 100:  
          sqlCursor.execute("INSERT INTO decks('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99') VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", card_expanded )
          sqlConnector.commit()
          responseRowId = sqlCursor.execute("SELECT last_insert_rowid()")
          rowId = responseRowId.fetchone()
          sqlCursor.execute("UPDATE deck_names SET fetched=1, id = ? WHERE name = ?", (rowId[0],dDeck[0][0]))
          sqlConnector.commit()
        else:
          sqlCursor.execute("DELETE FROM deck_names WHERE name = ?",(dDeck[0][0],))



def getDeckList(url):
  baseURL= "https://tappedout.net{url}?fmt=txt"
  return requests.get(baseURL.format(url=url)).text

def getDeckListMultithread(url):
  baseURL= "https://tappedout.net{url}?fmt=txt"
  return [url,requests.get(baseURL.format(url=url[0])).text]


def parseDeckList(dlist, sqlConnector):
  sqlCur = sqlConnector.cursor()
  deck = []
  for line in dlist.splitlines():
    line = line.strip()
    if len(line) == 0:
      continue

    if not line[0] in '0123456789':
      continue

    count = line.split(' ', maxsplit=1)[0]
    name = line.split(' ', maxsplit=1)[1]
    id = sqlCur.execute("SELECT ROWID FROM cards WHERE name= ?",(name,)).fetchone()
     
    idt = 0
    if id != None:
      idt = id[0]
    else:
      sqlCur.execute("INSERT INTO cards(name) VALUES(?)",(name,))
      sqlConnector.commit()
      id = sqlCur.execute("SELECT ROWID FROM cards WHERE name= ?",(name,)).fetchone()
      idt=id[0]


    card = [idt ,int(count), name ]
    deck.append(card)
  sqlCur.close()
  return deck


def saveDeckToSql(url, deck, sqlConnector):
  SQLcursor = sqlConnector.cursor()
  dbCheckResponse = SQLcursor.execute("SELECT * FROM deck_names WHERE name = ?", (url,))
  dbCheck = dbCheckResponse.fetchone()
  if dbCheck != []:
    card_expanded = []
    for card in deck:
      while card[1] != 0:
        card_expanded.append(card[0])
        card[1] = card[1]-1
    print(len(card_expanded))
    if len(card_expanded) == 100:  
      SQLcursor.execute("INSERT INTO decks('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99') VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", card_expanded )
      sqlConnector.commit()
      responseRowId = SQLcursor.execute("SELECT last_insert_rowid()")
      rowId = responseRowId.fetchone()
      SQLcursor.execute("INSERT INTO deck_names values (?,?,?)", (rowId[0], url, 1))
      sqlConnector.commit()
    
  SQLcursor.close()


def createDeckTables(sqlConnector):
  cur = sqlConnector.cursor()
  cur.execute("CREATE TABLE deck_names(id, name, fetched)")
  sqlConnector.commit()
  cur.execute("CREATE TABLE decks({seq})".format(seq=', '.join(map(lambda x: '"'+x+'"',map(str,range(0,100))))))
  sqlConnector.commit()
  cur.close()

def createCardTable(sqlConnectordeck, sqlConnectorprints):
  dCur = sqlConnectordeck.cursor()
  pCur = sqlConnectorprints.cursor()
  unr = pCur.execute("SELECT DISTINCT name FROM cards ")
  uniquenames = unr.fetchall()
  dCur.execute("CREATE TABLE cards (name TEXT) ")
  dCur.executemany("INSERT INTO cards(name) VALUES(?)", uniquenames)
  con.commit()
  dCur.close()
  pCur.close()

def clearDB(sqlConnector):
  cur = sqlConnector.cursor()
  cur.execute("DROP TABLE decks")
  con.commit()
  cur.execute("DROP TABLE deck_names")
  con.commit()
  cur.close()
  createDeckTables(sqlConnector)


In [151]:
fetchDecksMultithread(con)

0/2249
decklist: 10
100
100
101
100
100
100
100
100
100
100
10/2249
decklist: 10
100
100
100
100
100
100
100
100
100
100
20/2249
decklist: 10
100
100
102
100
100
100
100
100
100
100
30/2249
decklist: 10
100
100
100
100
100
100
100
100
100
101
40/2249
decklist: 10
100
100
100
101
100
105
100
100
100
100
50/2249
decklist: 10
100
100
100
104
100
100
100
140
100
100
60/2249
decklist: 10
100
100
100
100
100
100
100
100
100
100
70/2249
decklist: 10
100
100
100
104
100
100
108
100
100
100
80/2249
decklist: 10
100
100
100
100
100
100
100
100
101
100
90/2249
decklist: 10
100
100
101
100
100
100
100
100
100
108
100/2249
decklist: 10
100
100
100
108
100
100
110
127
110
100
110/2249
decklist: 10
127
100
132
100
100
102
100
100
100
100
120/2249
decklist: 10
100
100
100
100
100
100
100
101
108
115
130/2249
decklist: 10
100
100
100
100
100
100
100
100
100
100
140/2249
decklist: 10
100
100
100
100
100
100
100
100
100
100
150/2249
decklist: 10
0
100
100
100
100
100
100
100
100
100
160/2249
decklist: 10

KeyboardInterrupt: 

In [113]:
fetchDecks(con)

1/6397
100
2/6397
100
3/6397
100
4/6397
100
5/6397
100
6/6397
100
7/6397
100
8/6397
0
9/6397
100
10/6397
100
11/6397
100
12/6397
100
13/6397
100
14/6397
100
15/6397
100
16/6397
100
17/6397
100
18/6397
100
19/6397
117
20/6397
100
21/6397
100
22/6397
100
23/6397
100
24/6397
100
25/6397
100
26/6397
100
27/6397
100
28/6397
100
29/6397
100
30/6397
100
31/6397
100
32/6397
100
33/6397
100
34/6397
100
35/6397
100
36/6397
100
37/6397
100
38/6397
123
39/6397
100
40/6397
112
41/6397
0
42/6397
100
43/6397
100
44/6397
100
45/6397
100
46/6397
112
47/6397
100
48/6397
100
49/6397
100
50/6397
100
51/6397
100
52/6397
100
53/6397
100
54/6397
100
55/6397
100
56/6397
100
57/6397
100
58/6397
100
59/6397
100
60/6397
102
61/6397
100
62/6397
100
63/6397
100
64/6397
100
65/6397
100
66/6397
100
67/6397
100
68/6397
100
69/6397
100
70/6397
100
71/6397
100
72/6397
100
73/6397
100
74/6397
104
75/6397
100
76/6397
100
77/6397
100
78/6397
104
79/6397
100
80/6397
100
81/6397
100
82/6397
100
83/6397
136
84/6397
100
85/63

KeyboardInterrupt: 

In [67]:
cur = con.cursor()

In [5]:
"CREATE TABLE decks({seq})".format(seq=', '.join(map(lambda x: '"'+x+'"',map(str,range(0,100)))))

'CREATE TABLE decks("0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41", "42", "43", "44", "45", "46", "47", "48", "49", "50", "51", "52", "53", "54", "55", "56", "57", "58", "59", "60", "61", "62", "63", "64", "65", "66", "67", "68", "69", "70", "71", "72", "73", "74", "75", "76", "77", "78", "79", "80", "81", "82", "83", "84", "85", "86", "87", "88", "89", "90", "91", "92", "93", "94", "95", "96", "97", "98", "99")'

In [18]:
cur.execute("CREATE TABLE decks({seq})".format(seq=', '.join(map(lambda x: '"'+x+'"',map(str,range(0,100))))))

In [17]:
cur.execute("CREATE TABLE deck_names(id, name)")
con.commit()

In [ ]:
pr = prints.cursor()
re = pr.execute("SELECT id FROM cards WHERE name='Acidic Slime'")
lal = re.fetchone()



In [90]:
createCardTable(con,prints)
createDeckTables(con)

In [86]:
clearDB(con)

In [84]:
names = open('query.txt', 'r')
Lines = names.readlines()

In [96]:
for name in Lines:

 links = searchForLinks(q=name)
 saveLinks(links,con)


https://tappedout.net/mtg-decks/search/?q=big
&format=edh&price_min=&price_max=&o=-date_updated&submit=Filter+results&p=1&page=1
/mtg-decks/prismatic-dragonflight/
/mtg-decks/illuna-top-deck-mutate/
/mtg-decks/quandrix-quake/
/mtg-decks/an-old-school-nightmare/
/mtg-decks/10-07-22-sAH-my-gain-your-loss/
/mtg-decks/legion-maja-edh/
/mtg-decks/musashis-mosh-pit-primer/
/mtg-decks/26-08-22-ex-machina/
/mtg-decks/whats-under-jaliras-hat/
/mtg-decks/bloodline-of-an-ancient-coven/
/mtg-decks/the-snek-train/
/mtg-decks/29-08-19-jaN-chainer-reanimator/
/mtg-decks/call-of-krothulu/
/mtg-decks/curse-of-the-witchwood/
/mtg-decks/gimme-vito-gimme-fire-life-gain-is-what-i-desire/
/mtg-decks/the-cauldron-born-grimgrin/
/mtg-decks/greedy-ghost-and-nightmare-cat/
/mtg-decks/hamza-big-boys/
/mtg-decks/playing-lands-making-mana/
/mtg-decks/tiamat-horde-commandant/
https://tappedout.net/mtg-decks/search/?q=big
&format=edh&price_min=&price_max=&o=-date_updated&submit=Filter+results&p=2&page=2
/mtg-decks/m

In [87]:
for name in Lines:

 links = searchForLinks(q=name)
 for l in links:

  dlist = getDeckList(url=l)
  pdlist = parseDeckList(dlist=dlist, sqlConnector=con)
  
  saveDeckToSql(url=l,deck=pdlist,sqlConnector=con)



https://tappedout.net/mtg-decks/search/?q=big
&format=edh&price_min=&price_max=&o=-date_updated&submit=Filter+results&p=1&page=1
/mtg-decks/call-of-krothulu/
/mtg-decks/curse-of-the-witchwood/
/mtg-decks/gimme-vito-gimme-fire-life-gain-is-what-i-desire/
/mtg-decks/the-cauldron-born-grimgrin/
/mtg-decks/greedy-ghost-and-nightmare-cat/
/mtg-decks/hamza-big-boys/
/mtg-decks/playing-lands-making-mana/
/mtg-decks/tiamat-horde-commandant/
/mtg-decks/mayael-the-anima-big-stomp/
/mtg-decks/sidisi-presents-tales-from-the-crypt-40-budget/
/mtg-decks/right-tohatebear-arms-tymnasidar/
/mtg-decks/05-09-22-kynaios-and-tiro-of-meletis/
/mtg-decks/painbow-2-chromatic-boogaloo/
/mtg-decks/lumps-of-coal-for-everyone/
/mtg-decks/03-12-17-uncle-karlov/
/mtg-decks/thousand-blade-strike/
/mtg-decks/tymnaravos-hatredbears/
/mtg-decks/edh-hazezon/
/mtg-decks/sapling-fun/
/mtg-decks/sneak-attack-precon-to-upgrade/
https://tappedout.net/mtg-decks/search/?q=big
&format=edh&price_min=&price_max=&o=-date_updated&s

KeyboardInterrupt: 

In [ ]:
unc = prints.cursor()
unr = unc.execute("SELECT DISTINCT name FROM cards ")


uniquenames = unr.fetchall()
print(uniquenames)

In [ ]:
cur.execute("CREATE TABLE cards (name TEXT) ")

In [ ]:
cur.execute("CREATE TABLE cards (name TEXT) ")
cur.executemany("INSERT INTO cards(name) VALUES(?)", uniquenames)
con.commit()

In [ ]:
parseDeckList(dlist=dlist, sqlCur=cur)

In [25]:
cur.execute("DROP TABLE decks")
con.commit()
cur.execute("DROP TABLE deck_names")
con.commit()


In [82]:
cur = con.cursor()
cur.execute("SELECT * FROM decks")
selres = cur.fetchmany()
print(selres)
cur.close()

[(16345, 2264, 2180, 3065, 2704, 2688, 7375, 348, 5807, 976, 0, 4659, 1236, 3036, 5980, 2729, 2300, 6179, 6093, 6647, 20699, 11156, 1393, 368, 368, 368, 368, 5989, 15933, 0, 5887, 10656, 365, 365, 9391, 3616, 3616, 354, 611, 4150, 5862, 12124, 1192, 1246, 11210, 19830, 9832, 367, 367, 2358, 5992, 11466, 21047, 18758, 19568, 3320, 6288, 994, 339, 6279, 4421, 2622, 16863, 6142, 13333, 5659, 5941, 9087, 5995, 7838, 5403, 4104, 3294, 2662, 4541, 2751, 20370, 5429, 1993, 2753, 593, 7896, 5892, 2354, 2355, 12692, 11565, 7425, 5705, 6419, 9392, 612, 6080, 1115, 1007, 2578, 5646, 6399, 363, 6304)]


In [71]:
cur.close()
